In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))


In [3]:
## tqdm for loading bars
from tqdm import tqdm

import torch
## Torchvision
import torchvision
from torchvision import transforms
import SimpleITK as sitk
from multiprocessing import Pool

from src.utils.general import resample_volume

In [4]:
from src.models.simclr import SimCLR as SimCLRD
from src.data.self_supervised_dm import ContrastiveDataSet as ContrastiveDataSetD, ContrastiveDataModule

In [5]:
orig_path = Path('/mrhome/vladyslavz/git/central-sulcus-analysis/data/brainvisa_augm/nobackup/generated')
down_path = Path(orig_path = Path('/mrhome/vladyslavz/git/central-sulcus-analysis/data/brainvisa_augm_2x2x2/nobackup'))

In [6]:
imgs = [x for x in orig_path.glob('*/*_t1_*.nii.gz')]

In [7]:
len(imgs)

12400

In [8]:
def process_img(p):
    new_p = str(p).replace('brainvisa_augm', 'brainvisa_augm_2x2x2')
    Path(new_p).parent.mkdir(parents=True, exist_ok=True)
    img = sitk.ReadImage(str(p))
    
    image_interpolator = sitk.sitkNearestNeighbor if 'labels' in new_p else sitk.sitkLinear 
    res_img = resample_volume(img, [2, 2, 2], image_interpolator)
    sitk.WriteImage(res_img, new_p)
    return 1

In [9]:
# for p in tqdm(imgs):
#     process_img(p)

Use multiprocessing

In [10]:
with Pool(10) as p:
    r = list(tqdm(p.imap(process_img, imgs), total=len(imgs)))

100%|██████████| 12400/12400 [14:47<00:00, 13.97it/s]
